### Exploring the impact of clustering on the quality of SMOTE preprocessing. Comparative analysis
##### Maksym Malicki, Jacek Glapiński
###### Wrocław University of Technology
In this notebook we present a comparative analysis of the impact of clustering using various methods on the quality of SMOTE preprocessing.

#### load_dataset()
This method allows us to load datasets listed in the paper.

In [16]:
import numpy as np

def load_dataset(file_path):
    data = []
    labels = []

    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('@'):
                continue
            line_data = line.strip().split(',')
            sample_class = line_data[-1].strip().lower().replace(" ", "")
            label = 1 if sample_class == 'positive' else 0
            converted_data = []
            for x in line_data[:-1]:
                try:
                    converted_data.append(float(x))
                except ValueError:
                    converted_data.append(ord(x))
            data.append(converted_data)
            labels.append(label)
    X = np.array(data)
    y = np.array(labels)

    return X, y

#### Test implementacji SMOTE

##### IMPLEMENTACJA SMOTE

In [17]:
#https://medium.com/@corymaklin/synthetic-minority-over-sampling-technique-smote-7d419696b88c

from random import randrange, uniform
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score

def SMOTE_ByMedium(sample: np.array, N: int, k: int) -> np.array:
    
    T, num_attrs = sample.shape
    
    # If N is less than 100%, randomize the minority class samples as only a random percent of them will be SMOTEd
    if N < 100:
        T = round(N / 100 * T)
        N = 100
    # The amount of SMOTE is assumed to be in integral multiples of 100
    N = int(N / 100)

    synthetic = np.zeros([T * N, num_attrs])
    new_index = 0
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(sample.values)
    def populate(N, i, nnarray):
        
        nonlocal new_index
        nonlocal synthetic
        nonlocal sample
        while N != 0:
            nn = randrange(1, k+1)
            for attr in range(num_attrs):
                dif = sample.iloc[nnarray[nn]][attr] - sample.iloc[i][attr]
                gap = uniform(0, 1)
                synthetic[new_index][attr] = sample.iloc[i][attr] + gap * dif
            new_index += 1
            N = N - 1
    
    for i in range(T):
        nnarray = nbrs.kneighbors(sample.iloc[i].values.reshape(1, -1), return_distance=False)[0]
        populate(N, i, nnarray)
    
    return synthetic




In [18]:
#Smote test
# tutaj jest tymczasowy test Smote żeby ogarnąć poprawne liczebności
import plotly.express as px
temp_Cluter = np.array([[1,3,2,4,6,2,1,3,6,4,5,1,5,2,6,8,7,9,5,8,6,8,7,9,5,7,4,5,7],
                        [3,1,2,6,2,5,4,3,2,1,3,3,2,1,6,5,7,6,4,8,9,5,8,4,7,5,6,8,4]])
temp_Cluter_y = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1])
print('temp_Cluter shape: ',temp_Cluter.shape)
print('temp_Cluter_y shape: ',temp_Cluter_y.shape)
fig = px.scatter(x=temp_Cluter[0],y=temp_Cluter[1],title="Dane wejściowe")
fig.show()

temp_Cluter shape:  (2, 29)
temp_Cluter_y shape:  (29,)


Liczebność zbioru wejściowego:

In [19]:
import math
# teraz określamy ile chcemy próbek dorobić:
# chcemy mieć 100 próbek
desired = 100
print('Chcemy uzyskać 100 próbek wyjściowych')
#obecnie mamy size próbek
size  = temp_Cluter.shape[1]

todoSamples = int(((desired/size)-1 )*100)
print("Przewidujemy, że dosaniemy próbek wyjściowych: ", (todoSamples/100+1)*len(temp_Cluter_y))
print((todoSamples/100+1))
print(math.floor(todoSamples/100)+1)
temp_out = SMOTE_ByMedium(pd.DataFrame(temp_Cluter.T), todoSamples, 5)

Chcemy uzyskać 100 próbek wyjściowych
Przewidujemy, że dosaniemy próbek wyjściowych:  99.76
3.44
3


In [20]:
temp_out.T
print(len(temp_out.T[0]))
print(len(temp_Cluter[0]))
print(len(temp_out.T[0])+len(temp_Cluter[0]))

58
29
87


In [21]:
temp_Cluter_OUT = [[*temp_Cluter[0],*temp_out.T[0]],[*temp_Cluter[1],*temp_out.T[1]]]
print(temp_Cluter_OUT)

[[1, 3, 2, 4, 6, 2, 1, 3, 6, 4, 5, 1, 5, 2, 6, 8, 7, 9, 5, 8, 6, 8, 7, 9, 5, 7, 4, 5, 7, 1.0, 2.4659050830554667, 3.8834101177006404, 3.7978059791521925, 2.0, 2.278774764302708, 3.1543656464519554, 4.524298563064091, 5.758421699385621, 5.832697882819067, 1.70542539465129, 3.8167256433954937, 1.7704672366715455, 1.0, 1.3226397786545068, 2.2789744621524637, 6.0, 5.88383711896199, 3.6394542351975363, 5.083146115785436, 5.0, 5.0, 1.0, 1.6003562477007223, 5.0, 5.642893830366326, 1.1924307655803756, 2.0, 6.383944440845423, 5.725292001044635, 8.49138125793966, 8.843237494846393, 7.556470810273051, 7.0, 9.0, 8.412894728091016, 5.0, 5.0, 7.347528232090392, 7.046745818273109, 6.762941686735793, 6.778106879698074, 7.934489165670645, 7.606922531788311, 6.414450862883358, 6.571729661222177, 9.0, 8.27009924025273, 5.0, 4.290480352378447, 7.0, 7.587995768830378, 4.619855554747724, 4.0, 6.300090064473732, 5.0, 7.293509333871607, 7.0], [3, 1, 2, 6, 2, 5, 4, 3, 2, 1, 3, 3, 2, 1, 6, 5, 7, 6, 4, 8, 9, 5, 

In [22]:
fig = px.scatter(x= temp_out.T[0], y= temp_out.T[1],title='Wygenerowane')
fig.show()

In [23]:
fig = px.scatter(x= temp_Cluter_OUT[0], y= temp_Cluter_OUT[1], title= 'Wygenerowane + wejściowe o łącznej liczbie równej desired')
fig.show()
len(temp_Cluter_OUT[0])

87

#### Clustering with SMOTE (Część pracy)

In [24]:
from sklearn.cluster import KMeans, MeanShift

def oversample_clustered_data(X, y, X_minority, y_minority, X_majority, y_majority, cluster_labeled_data):
    #wygenerowane próbki mniejszościowe wrzucamy tutaj:
    X_generated = []

    # Do oceny ile próbek trzeba będzie wygenerować trzeba przeliczyć różnicę między liczebnością klasy mniejszościowej i większościowej, będzie to potrzebne później w pętli dla każego z klastrów
    majority_minority_difference =  list(y).count(0) - list(y).count(1)
    if(majority_minority_difference < 0):
        print("[ERR] Monority class has grater count to Majority class")
        exit()
    # Do oceny ile próbek będzie trzeba wygenerować potrzebne też będzie informacja o liczbności klasy mniejszościowej
    minority_count = list(y).count(1)

    # Dla każdego z klastrów:
    cluster_labels = np.unique(cluster_labeled_data) #  pobranie info jakie mamy labele klastrów [0,1,...]

    # Wszystkie wygenerowane syntetycznie obiekty trafią tutaj
    syntetic_data = []
    for cluster in cluster_labels:
        # w pierwszej kolejności filtrujemy indeksy zbioru mniejszościowego aby pozostawić tylko indeksy należące do klastra który w danej iteracji pętli analizujemy
        cluster_samples_indices_minority = np.where(cluster_labeled_data == cluster)[0]
        # ustalamy ile próbek trzeba wygenerować w tym klastrze
        # ustalamy procentowo liczebność (udział) klastra względem całej klasy mniejszościowej
        percentage_of_count = len(cluster_samples_indices_minority)/minority_count
        # zabezpieczenie jeśli coś na tym etapie szłoby bardzo nie tak
        if(percentage_of_count>1 or percentage_of_count<0):
            print("Kurza twarz.. something is no yes! Procenty nie mogą być większe niż 100 ani mniejsze niż 0")
            exit()
        
        #następnie przemnarzamy udział procentowy klastra przez różnicę między zbiorem większościowym i mniejszościowym uzyskując ilość próbek do augmentacji
        num_of_samples_to_generate_in_this_cluster = percentage_of_count * majority_minority_difference

        # Do samej funkcji augmentacji wymagana jest warość procentowa gdzie 100 to 100% określająca ile próbek ma zostać wygenerowanych względem danych wejściowych
        # w tym celu należy podzielić ilość próbek które chcemy wygenerować przez liczbę próbek które posiadamy już
        todoSamples = 100*(num_of_samples_to_generate_in_this_cluster/len(cluster_samples_indices_minority))

        # Teraz musimy niestety przejść z np.array na dataframe bo w ten sposób działa funkcja ale robimy to już w parametrze funkcji
        # Oversampling... totaj k = 5.. jest to hiperparametr który nie wydaje się być istotny w tej implementacji. Jeśli będą większe zmiany trzeba spojrzeć na niego
        temp_out = SMOTE_ByMedium(pd.DataFrame(X_minority), todoSamples, 5)

        #Ta implementacja Smote jeśli tworzy więcej niż 100% próbek z próbek które dostała traci informację o części dziesiętnej procentów - to powoduje, że powstaje mniej resamplowanych próbek
        if todoSamples>100:
            todoSamples_2 = todoSamples%100
            temp_out_2 = SMOTE_ByMedium(pd.DataFrame(X_minority),todoSamples_2 , 5)
        syntetic_data.append(temp_out)
        syntetic_data.append(temp_out_2)

    #tutaj wymagane jest połączenie wyjścia syntetic_data razem z X i y
    X_resampled = X
    y_resampled  = y

    for clusterOUT in syntetic_data:
        X_resampled = np.block([[X_resampled], [clusterOUT]])
        y_resampled = [*y_resampled,*np.ones(clusterOUT.shape[0])]# jedynki dla tego, że 1 to klasa mniejszościowa
    return X_resampled, y_resampled

def KMeans_SMOTE(X, y, num_clusters):
    # Wybieramy indeksy klasy mniejszościowej i większościowej
    minority_indices = np.where(y == 1)[0] # dla naszych datasetów klasa o labelu 1 zawsze jest mniejszościowa
    majority_indices = np.where(y == 0)[0]

    # Sortujemy Dane i Labele na minority i majority
    X_minority = X[minority_indices]
    y_minority = y[minority_indices]

    X_majority = X[majority_indices]
    y_majority = y[majority_indices]
    
    # Wykonujemy klasteryzację na klasie mniejszościowej
    kmeans_labels_minority = KMeans(n_clusters=num_clusters, random_state=0, n_init="auto").fit_predict(X_minority)

    # Zwrócona wartość to indeksy odwołujące się jedynie do klasy mniejszościowej! trzeba o tym teraz pamiętać.
    return oversample_clustered_data(X, y, X_minority, y_minority, X_majority, y_majority, kmeans_labels_minority)


def MeanShift_SMOTE(X, y):
    # TAK WIEM,TO POWINNA BYĆ FUNKCJA, BO WIĘKSZOŚĆ LINIJEK MEANSHIFT_SMOTE I KMEANS_SMOTE SIĘ POWTARZA ... może kiedyś tu upiększymy
    # Wybieramy indeksy klasy mniejszościowej i większościowej
    minority_indices = np.where(y == 1)[0] # dla naszych datasetów klasa o labelu 1 zawsze jest mniejszościowa
    majority_indices = np.where(y == 0)[0]

    # Sortujemy Dane i Labele na minority i majority
    X_minority = X[minority_indices]
    y_minority = y[minority_indices]

    X_majority = X[majority_indices]
    y_majority = y[majority_indices]
    
    # Wykonujemy klasteryzację na klasie mniejszościowej
    mean_shift_labels_minority = MeanShift().fit_predict(X_minority)
    return oversample_clustered_data(X, y, X_minority, y_minority, X_majority, y_majority, mean_shift_labels_minority)

#### Sprawdzenie który ze zbiorów jest Większościowy

In [25]:
# tymczasowy algorytm do analizy która z klas jest mniejszościowa
def ReturnsMinorityLabel(file_path):
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('@'):
                continue
            line_data = line.strip().split(',')
            sample_class = line_data[-1].strip().lower().replace(" ", "")
            label = 1 if sample_class == 'positive' else 0
            labels.append(label)
    countOf0 = labels.count(0)
    countOf1 = labels.count(1)
    print("---------STATS---------") 
    print('countOf0: ',countOf0)
    print('countOf1: ',countOf1)
    print("----------OUT----------") 
    if countOf0 == countOf1:
        print("!!!!!kurcze zbiorki są zbalansowane!!!!!!")
        return(-1)
    if countOf0 > countOf1:
        print("Więcej klasy 0")
        return(0)
    if countOf0 < countOf1:
        print("Więcej klasy 1")
        return(1)

import os

#sprawdzenie czy działa

directories = ['mild-imbalance', 'high-imbalance']
results_of_test = []
for directory in directories:
    print(f"Processing files in directory: {directory}")
    files = os.listdir(directory)
    
    for file_name in files:
        file_path = os.path.join(directory, file_name)
        print(f"File: {file_path}")
        results_of_test.append(ReturnsMinorityLabel(file_path))
    
print(results_of_test)
print(results_of_test.count(1))

Processing files in directory: mild-imbalance
File: mild-imbalance\page-blocks0.dat
---------STATS---------
countOf0:  4913
countOf1:  559
----------OUT----------
Więcej klasy 0
File: mild-imbalance\pima.dat
---------STATS---------
countOf0:  500
countOf1:  268
----------OUT----------
Więcej klasy 0
File: mild-imbalance\segment0.dat
---------STATS---------
countOf0:  1979
countOf1:  329
----------OUT----------
Więcej klasy 0
File: mild-imbalance\vehicle0.dat
---------STATS---------
countOf0:  647
countOf1:  199
----------OUT----------
Więcej klasy 0
File: mild-imbalance\vehicle1.dat
---------STATS---------
countOf0:  629
countOf1:  217
----------OUT----------
Więcej klasy 0
File: mild-imbalance\vehicle2.dat
---------STATS---------
countOf0:  628
countOf1:  218
----------OUT----------
Więcej klasy 0
File: mild-imbalance\vehicle3.dat
---------STATS---------
countOf0:  634
countOf1:  212
----------OUT----------
Więcej klasy 0
File: mild-imbalance\wisconsin.dat
---------STATS---------
coun

#### Experiment for single dataset

In [27]:
from imblearn.over_sampling import SMOTE, RandomOverSampler, BorderlineSMOTE
from sklearn import svm
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_score, recall_score
from imblearn.metrics import specificity_score

def experiment(X, y):
    preprocessings = {
        "KMeansSMOTE": True,
        "MeansShiftSMOTE": True,
        "SMOTE": SMOTE(),
        "ROS": RandomOverSampler(),
        "BorderlineSMOTE": BorderlineSMOTE(),
    }
    classifier = RandomForestClassifier(random_state=42)
    rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1234)
    result = {}
    for key in preprocessings:
        precision_scores = []
        recall_scores = []
        specifity_scores = []
        for train_index, test_index in rskf.split(X,y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            if key == "KMeansSMOTE":
                X_train_oversampled, y_train_oversampled = KMeans_SMOTE(X_train, y_train, 2)
            elif key == "MeansShiftSMOTE":
                X_train_oversampled, y_train_oversampled = MeanShift_SMOTE(X_train, y_train)
            else:
                X_train_oversampled, y_train_oversampled = preprocessings[key].fit_resample(X_train, y_train)
            classifier.fit(X_train_oversampled, y_train_oversampled)
            predict = classifier.predict(X_test)
            precision_scores.append(precision_score(y_test, predict))
            recall_scores.append(recall_score(y_test, predict))
            specifity_scores.append(specificity_score(y_test, predict))
        mean_precision_score = np.mean(precision_scores)
        std_precision_score = np.std(precision_scores)
        mean_recall_score = np.mean(recall_scores)
        std_recall_score = np.std(recall_scores)
        mean_specifity_score = np.mean(specifity_scores)
        std_specifity_score = np.std(specifity_scores)
#         print(f"Precission score {key}: %.3f (%.3f)" % (mean_precision_score, std_precision_score))
#         print(f"Specifity score {key}: %.3f (%.3f)" % (mean_specifity_score, std_specifity_score))
#         print(f"Recall score {key}: %.3f (%.3f)" % (mean_recall_score, std_recall_score))
        result[key] = {
            "precission_scores": precision_scores,
            "recall_scores": recall_scores,
            "specifity_scores": specifity_scores,
            "mean_precission_score": mean_precision_score,
            "mean_recall_scores": mean_recall_score,
            "mean_specifity_scores": mean_specifity_score,
        }
    return result

#### Running experiments on the datasets

In [28]:
import os

directories = ['mild-imbalance', 'high-imbalance']
results = {}
for directory in directories:
    print(f"Processing files in directory: {directory}")
    files = os.listdir(directory)
    results = {}
    for file_name in files:
        file_path = os.path.join(directory, file_name)
        print(f"File: {file_path}")
        X, y = load_dataset(file_path)
        experiment_result = experiment(X, y)
        results[file_name] = experiment_result
# print(results)

Processing files in directory: mild-imbalance
File: mild-imbalance\page-blocks0.dat


#### Uproszczony eksperyment na pojedynczych danych banchmarkowych

In [14]:

from imblearn.datasets import fetch_datasets

satimage = fetch_datasets()["satimage"]
X, y = satimage.data, satimage.target

def replace_negatives_with_zeros(arr):
    return [0 if x == -1 else x for x in arr]
y = replace_negatives_with_zeros(y)
experiment_result = experiment(X, y)
results[file_name] = experiment_result

TypeError: only integer scalar arrays can be converted to a scalar index

### Statistically significantly better preprocessings in given datasets with given metrics

In [ ]:
results = {'abalone-17_vs_7-8-9-10.dat': {'KMeansSMOTE': {'precission_scores': [0.19444444444444445,
    0.24242424242424243,
    0.20408163265306123,
    0.20512820512820512,
    0.2631578947368421,
    0.17857142857142858,
    0.21875,
    0.17647058823529413,
    0.14285714285714285,
    0.17647058823529413],
   'recall_scores': [0.5833333333333334,
    0.6666666666666666,
    0.8333333333333334,
    0.7272727272727273,
    0.45454545454545453,
    0.4166666666666667,
    0.5833333333333334,
    0.5,
    0.5454545454545454,
    0.5454545454545454],
   'specifity_scores': [0.9364035087719298,
    0.9451754385964912,
    0.9144736842105263,
    0.9320175438596491,
    0.9692982456140351,
    0.9495614035087719,
    0.9451754385964912,
    0.9385964912280702,
    0.9210526315789473,
    0.9385964912280702],
   'mean_precission_score': 0.20023561672859547,
   'mean_recall_scores': 0.5856060606060606,
   'mean_specifity_scores': 0.9390350877192983},
  'MeansShiftSMOTE': {'precission_scores': [0.15384615384615385,
    0.25,
    0.16,
    0.17073170731707318,
    0.23076923076923078,
    0.15384615384615385,
    0.25,
    0.16666666666666666,
    0.14285714285714285,
    0.15],
   'recall_scores': [0.5,
    0.75,
    0.6666666666666666,
    0.6363636363636364,
    0.5454545454545454,
    0.5,
    0.6666666666666666,
    0.5833333333333334,
    0.5454545454545454,
    0.5454545454545454],
   'specifity_scores': [0.9276315789473685,
    0.9407894736842105,
    0.9078947368421053,
    0.9254385964912281,
    0.956140350877193,
    0.9276315789473685,
    0.9473684210526315,
    0.9232456140350878,
    0.9210526315789473,
    0.9254385964912281],
   'mean_precission_score': 0.1828717055302421,
   'mean_recall_scores': 0.5939393939393938,
   'mean_specifity_scores': 0.9302631578947368},
  'SMOTE': {'precission_scores': [0.30434782608695654,
    0.2777777777777778,
    0.2608695652173913,
    0.28,
    0.2727272727272727,
    0.36363636363636365,
    0.2777777777777778,
    0.29411764705882354,
    0.25,
    0.25],
   'recall_scores': [0.5833333333333334,
    0.4166666666666667,
    0.5,
    0.6363636363636364,
    0.2727272727272727,
    0.3333333333333333,
    0.4166666666666667,
    0.4166666666666667,
    0.5454545454545454,
    0.36363636363636365],
   'specifity_scores': [0.9649122807017544,
    0.9714912280701754,
    0.9627192982456141,
    0.9605263157894737,
    0.9824561403508771,
    0.9846491228070176,
    0.9714912280701754,
    0.9736842105263158,
    0.9605263157894737,
    0.9736842105263158],
   'mean_precission_score': 0.28312542302823634,
   'mean_recall_scores': 0.44848484848484843,
   'mean_specifity_scores': 0.9706140350877194},
  'ROS': {'precission_scores': [0.5,
    0.5,
    1.0,
    0.3333333333333333,
    0.5,
    1.0,
    0.5,
    0.4,
    0.42857142857142855,
    0.3333333333333333],
   'recall_scores': [0.16666666666666666,
    0.25,
    0.16666666666666666,
    0.18181818181818182,
    0.09090909090909091,
    0.08333333333333333,
    0.08333333333333333,
    0.16666666666666666,
    0.2727272727272727,
    0.18181818181818182],
   'specifity_scores': [0.9956140350877193,
    0.993421052631579,
    1.0,
    0.9912280701754386,
    0.9978070175438597,
    1.0,
    0.9978070175438597,
    0.993421052631579,
    0.9912280701754386,
    0.9912280701754386],
   'mean_precission_score': 0.5495238095238094,
   'mean_recall_scores': 0.1643939393939394,
   'mean_specifity_scores': 0.9951754385964913},
  'BorderlineSMOTE': {'precission_scores': [0.3157894736842105,
    0.3333333333333333,
    0.25,
    0.3,
    0.3333333333333333,
    0.2222222222222222,
    0.26666666666666666,
    0.4,
    0.29411764705882354,
    0.1875],
   'recall_scores': [0.5,
    0.4166666666666667,
    0.3333333333333333,
    0.5454545454545454,
    0.2727272727272727,
    0.16666666666666666,
    0.3333333333333333,
    0.5,
    0.45454545454545453,
    0.2727272727272727],
   'specifity_scores': [0.9714912280701754,
    0.9780701754385965,
    0.9736842105263158,
    0.9692982456140351,
    0.9868421052631579,
    0.9846491228070176,
    0.9758771929824561,
    0.9802631578947368,
    0.9736842105263158,
    0.9714912280701754],
   'mean_precission_score': 0.29029626762985894,
   'mean_recall_scores': 0.3795454545454545,
   'mean_specifity_scores': 0.9765350877192981}},
 'abalone19.dat': {'KMeansSMOTE': {'precission_scores': [0.030303030303030304,
    0.03508771929824561,
    0.022222222222222223,
    0.03508771929824561,
    0.05555555555555555,
    0.0,
    0.0,
    0.06779661016949153,
    0.08163265306122448,
    0.027777777777777776],
   'recall_scores': [0.3333333333333333,
    0.3333333333333333,
    0.14285714285714285,
    0.2857142857142857,
    0.5,
    0.0,
    0.0,
    0.5714285714285714,
    0.5714285714285714,
    0.3333333333333333],
   'specifity_scores': [0.9227985524728589,
    0.9336550060313631,
    0.9468599033816425,
    0.9335748792270532,
    0.9384057971014492,
    0.9324487334137516,
    0.9433051869722557,
    0.9335748792270532,
    0.9456521739130435,
    0.9154589371980676],
   'mean_precission_score': 0.03554632876857931,
   'mean_recall_scores': 0.30714285714285716,
   'mean_specifity_scores': 0.9345734048938539},
  'MeansShiftSMOTE': {'precission_scores': [0.015625,
    0.04054054054054054,
    0.01818181818181818,
    0.03125,
    0.04054054054054054,
    0.0,
    0.0,
    0.0547945205479452,
    0.05714285714285714,
    0.023809523809523808],
   'recall_scores': [0.16666666666666666,
    0.5,
    0.14285714285714285,
    0.2857142857142857,
    0.5,
    0.0,
    0.0,
    0.5714285714285714,
    0.5714285714285714,
    0.3333333333333333],
   'specifity_scores': [0.9240048250904704,
    0.9143546441495778,
    0.9347826086956522,
    0.9251207729468599,
    0.9142512077294686,
    0.9203860072376358,
    0.9433051869722557,
    0.9166666666666666,
    0.9202898550724637,
    0.9009661835748792],
   'mean_precission_score': 0.02818848007632254,
   'mean_recall_scores': 0.30714285714285716,
   'mean_specifity_scores': 0.9214127958135929},
  'SMOTE': {'precission_scores': [0.05555555555555555,
    0.08333333333333333,
    0.07692307692307693,
    0.0,
    0.08333333333333333,
    0.0,
    0.0,
    0.16666666666666666,
    0.10526315789473684,
    0.0],
   'recall_scores': [0.16666666666666666,
    0.3333333333333333,
    0.14285714285714285,
    0.0,
    0.16666666666666666,
    0.0,
    0.0,
    0.42857142857142855,
    0.2857142857142857,
    0.0],
   'specifity_scores': [0.9794933655006032,
    0.9734620024125452,
    0.9855072463768116,
    0.9879227053140096,
    0.9867149758454107,
    0.9794933655006032,
    0.9722557297949337,
    0.9818840579710145,
    0.9794685990338164,
    0.9746376811594203],
   'mean_precission_score': 0.057107512370670264,
   'mean_recall_scores': 0.15238095238095237,
   'mean_specifity_scores': 0.9800839728909168},
  'ROS': {'precission_scores': [0.0,
    0.0,
    1.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    1.0,
    0.0],
   'recall_scores': [0.0,
    0.0,
    0.14285714285714285,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.14285714285714285,
    0.0],
   'specifity_scores': [1.0,
    1.0,
    1.0,
    0.998792270531401,
    1.0,
    0.9987937273823885,
    1.0,
    1.0,
    1.0,
    0.998792270531401],
   'mean_precission_score': 0.2,
   'mean_recall_scores': 0.02857142857142857,
   'mean_specifity_scores': 0.9996378268445192},
  'BorderlineSMOTE': {'precission_scores': [0.0,
    0.1,
    0.125,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.2,
    0.0],
   'recall_scores': [0.0,
    0.16666666666666666,
    0.14285714285714285,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.14285714285714285,
    0.0],
   'specifity_scores': [0.9951749095295537,
    0.9891435464414958,
    0.9915458937198067,
    0.998792270531401,
    0.9975845410628019,
    1.0,
    0.9903498190591074,
    0.9939613526570048,
    0.9951690821256038,
    0.9951690821256038],
   'mean_precission_score': 0.0425,
   'mean_recall_scores': 0.04523809523809524,
   'mean_specifity_scores': 0.994689049725238}},
 'kr-vs-k-three_vs_eleven.dat': {'KMeansSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    0.9411764705882353],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 0.9941176470588236,
   'mean_specifity_scores': 1.0},
  'MeansShiftSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'SMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    0.8823529411764706],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 0.9882352941176471,
   'mean_specifity_scores': 1.0},
  'ROS': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    0.9411764705882353],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 0.9941176470588236,
   'mean_specifity_scores': 1.0},
  'BorderlineSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    0.8823529411764706],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 0.9882352941176471,
   'mean_specifity_scores': 1.0}},
 'kr-vs-k-zero-one_vs_draw.dat': {'KMeansSMOTE': {'precission_scores': [1.0,
    0.9473684210526315,
    0.9090909090909091,
    0.9545454545454546,
    1.0,
    0.9523809523809523,
    0.8571428571428571,
    1.0,
    1.0,
    1.0],
   'recall_scores': [0.9523809523809523,
    0.8571428571428571,
    0.9523809523809523,
    1.0,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571],
   'specifity_scores': [1.0,
    0.998211091234347,
    0.9964221824686941,
    0.998211091234347,
    1.0,
    0.9982142857142857,
    0.9946332737030411,
    1.0,
    1.0,
    1.0],
   'mean_precission_score': 0.9620528594212804,
   'mean_recall_scores': 0.9285714285714286,
   'mean_specifity_scores': 0.9985691924354715},
  'MeansShiftSMOTE': {'precission_scores': [1.0,
    0.9473684210526315,
    0.9090909090909091,
    0.9545454545454546,
    1.0,
    0.9523809523809523,
    0.8636363636363636,
    1.0,
    1.0,
    0.9473684210526315],
   'recall_scores': [0.9523809523809523,
    0.8571428571428571,
    0.9523809523809523,
    1.0,
    0.9523809523809523,
    0.9523809523809523,
    0.9047619047619048,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571],
   'specifity_scores': [1.0,
    0.998211091234347,
    0.9964221824686941,
    0.998211091234347,
    1.0,
    0.9982142857142857,
    0.9946332737030411,
    1.0,
    1.0,
    0.998211091234347],
   'mean_precission_score': 0.9574390521758943,
   'mean_recall_scores': 0.9333333333333333,
   'mean_specifity_scores': 0.9983903015589062},
  'SMOTE': {'precission_scores': [1.0,
    0.9473684210526315,
    0.9090909090909091,
    0.9545454545454546,
    1.0,
    0.9523809523809523,
    0.9,
    1.0,
    1.0,
    1.0],
   'recall_scores': [0.9523809523809523,
    0.8571428571428571,
    0.9523809523809523,
    1.0,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571],
   'specifity_scores': [1.0,
    0.998211091234347,
    0.9964221824686941,
    0.998211091234347,
    1.0,
    0.9982142857142857,
    0.9964221824686941,
    1.0,
    1.0,
    1.0],
   'mean_precission_score': 0.9663385737069948,
   'mean_recall_scores': 0.9285714285714286,
   'mean_specifity_scores': 0.9987480833120369},
  'ROS': {'precission_scores': [0.9545454545454546,
    0.95,
    0.9523809523809523,
    0.9545454545454546,
    1.0,
    0.9545454545454546,
    0.8636363636363636,
    1.0,
    1.0,
    0.9444444444444444],
   'recall_scores': [1.0,
    0.9047619047619048,
    0.9523809523809523,
    1.0,
    0.9523809523809523,
    1.0,
    0.9047619047619048,
    0.9523809523809523,
    0.9523809523809523,
    0.8095238095238095],
   'specifity_scores': [0.9982142857142857,
    0.998211091234347,
    0.998211091234347,
    0.998211091234347,
    1.0,
    0.9982142857142857,
    0.9946332737030411,
    1.0,
    1.0,
    0.998211091234347],
   'mean_precission_score': 0.9574098124098125,
   'mean_recall_scores': 0.9428571428571428,
   'mean_specifity_scores': 0.9983906210069001},
  'BorderlineSMOTE': {'precission_scores': [1.0,
    0.9411764705882353,
    0.9090909090909091,
    0.9545454545454546,
    0.9090909090909091,
    0.9523809523809523,
    0.8636363636363636,
    1.0,
    1.0,
    1.0],
   'recall_scores': [0.9523809523809523,
    0.7619047619047619,
    0.9523809523809523,
    1.0,
    0.9523809523809523,
    0.9523809523809523,
    0.9047619047619048,
    0.9523809523809523,
    0.9523809523809523,
    0.8571428571428571],
   'specifity_scores': [1.0,
    0.998211091234347,
    0.9964221824686941,
    0.998211091234347,
    0.9964221824686941,
    0.9982142857142857,
    0.9946332737030411,
    1.0,
    1.0,
    1.0],
   'mean_precission_score': 0.9529921059332824,
   'mean_recall_scores': 0.9238095238095239,
   'mean_specifity_scores': 0.998211410682341}},
 'shuttle-2_vs_5.dat': {'KMeansSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'MeansShiftSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'SMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'ROS': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'BorderlineSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0}},
 'shuttle-c0-vs-c4.dat': {'KMeansSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'MeansShiftSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'SMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'ROS': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0},
  'BorderlineSMOTE': {'precission_scores': [1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0,
    1.0],
   'recall_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'specifity_scores': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
   'mean_precission_score': 1.0,
   'mean_recall_scores': 1.0,
   'mean_specifity_scores': 1.0}},
 'yeast-0-2-5-6_vs_3-7-8-9.dat': {'KMeansSMOTE': {'precission_scores': [0.5,
    0.6521739130434783,
    0.5,
    0.4827586206896552,
    0.5714285714285714,
    0.6470588235294118,
    0.5,
    0.5769230769230769,
    0.5185185185185185,
    0.47619047619047616],
   'recall_scores': [0.65,
    0.75,
    0.6,
    0.7,
    0.631578947368421,
    0.55,
    0.65,
    0.75,
    0.7,
    0.5263157894736842],
   'specifity_scores': [0.9281767955801105,
    0.9558011049723757,
    0.9337016574585635,
    0.9171270718232044,
    0.9502762430939227,
    0.9668508287292817,
    0.9281767955801105,
    0.9392265193370166,
    0.9281767955801105,
    0.9392265193370166],
   'mean_precission_score': 0.5425052000323187,
   'mean_recall_scores': 0.6507894736842105,
   'mean_specifity_scores': 0.9386740331491712},
  'MeansShiftSMOTE': {'precission_scores': [0.4482758620689655,
    0.5,
    0.42857142857142855,
    0.42424242424242425,
    0.3793103448275862,
    0.5454545454545454,
    0.38235294117647056,
    0.48484848484848486,
    0.41935483870967744,
    0.39285714285714285],
   'recall_scores': [0.65,
    0.75,
    0.6,
    0.7,
    0.5789473684210527,
    0.6,
    0.65,
    0.8,
    0.65,
    0.5789473684210527],
   'specifity_scores': [0.9116022099447514,
    0.9171270718232044,
    0.9116022099447514,
    0.8950276243093923,
    0.9005524861878453,
    0.9447513812154696,
    0.8839779005524862,
    0.9060773480662984,
    0.9005524861878453,
    0.9060773480662984],
   'mean_precission_score': 0.44052680127567256,
   'mean_recall_scores': 0.6557894736842107,
   'mean_specifity_scores': 0.9077348066298343},
  'SMOTE': {'precission_scores': [0.5909090909090909,
    0.7647058823529411,
    0.6666666666666666,
    0.5909090909090909,
    0.6470588235294118,
    0.7692307692307693,
    0.65,
    0.6521739130434783,
    0.7058823529411765,
    0.55],
   'recall_scores': [0.65,
    0.65,
    0.5,
    0.65,
    0.5789473684210527,
    0.5,
    0.65,
    0.75,
    0.6,
    0.5789473684210527],
   'specifity_scores': [0.9502762430939227,
    0.9779005524861878,
    0.9723756906077348,
    0.9502762430939227,
    0.9668508287292817,
    0.9834254143646409,
    0.9613259668508287,
    0.9558011049723757,
    0.9723756906077348,
    0.9502762430939227],
   'mean_precission_score': 0.6587536589582625,
   'mean_recall_scores': 0.6107894736842105,
   'mean_specifity_scores': 0.9640883977900552},
  'ROS': {'precission_scores': [0.75,
    0.9090909090909091,
    0.6923076923076923,
    0.6666666666666666,
    0.75,
    0.75,
    0.8,
    0.8666666666666667,
    0.75,
    0.6111111111111112],
   'recall_scores': [0.6,
    0.5,
    0.45,
    0.6,
    0.47368421052631576,
    0.3,
    0.6,
    0.65,
    0.6,
    0.5789473684210527],
   'specifity_scores': [0.9779005524861878,
    0.994475138121547,
    0.9779005524861878,
    0.9668508287292817,
    0.9834254143646409,
    0.988950276243094,
    0.9834254143646409,
    0.988950276243094,
    0.9779005524861878,
    0.9613259668508287],
   'mean_precission_score': 0.7545843045843046,
   'mean_recall_scores': 0.5352631578947369,
   'mean_specifity_scores': 0.9801104972375692},
  'BorderlineSMOTE': {'precission_scores': [0.5454545454545454,
    0.8666666666666667,
    0.6428571428571429,
    0.6190476190476191,
    0.5625,
    0.7272727272727273,
    0.6666666666666666,
    0.5909090909090909,
    0.65,
    0.6666666666666666],
   'recall_scores': [0.6,
    0.65,
    0.45,
    0.65,
    0.47368421052631576,
    0.4,
    0.6,
    0.65,
    0.65,
    0.5263157894736842],
   'specifity_scores': [0.9447513812154696,
    0.988950276243094,
    0.9723756906077348,
    0.9558011049723757,
    0.9613259668508287,
    0.9834254143646409,
    0.9668508287292817,
    0.9502762430939227,
    0.9613259668508287,
    0.9723756906077348],
   'mean_precission_score': 0.6538041125541126,
   'mean_recall_scores': 0.5650000000000001,
   'mean_specifity_scores': 0.9657458563535911}},
 'yeast-0-2-5-7-9_vs_3-6-8.dat': {'KMeansSMOTE': {'precission_scores': [0.6818181818181818,
    0.7619047619047619,
    0.75,
    0.6428571428571429,
    0.7894736842105263,
    0.6956521739130435,
    0.9,
    0.7777777777777778,
    0.782608695652174,
    0.7083333333333334],
   'recall_scores': [0.75,
    0.8,
    0.9,
    0.9,
    0.7894736842105263,
    0.8,
    0.9,
    0.7,
    0.9,
    0.8947368421052632],
   'specifity_scores': [0.9613259668508287,
    0.9723756906077348,
    0.9668508287292817,
    0.9447513812154696,
    0.9779005524861878,
    0.9613259668508287,
    0.988950276243094,
    0.9779005524861878,
    0.9723756906077348,
    0.9613259668508287],
   'mean_precission_score': 0.7490425751466941,
   'mean_recall_scores': 0.833421052631579,
   'mean_specifity_scores': 0.9685082872928177},
  'MeansShiftSMOTE': {'precission_scores': [0.64,
    0.7619047619047619,
    0.75,
    0.6666666666666666,
    0.6666666666666666,
    0.625,
    0.8260869565217391,
    0.7142857142857143,
    0.7391304347826086,
    0.6666666666666666],
   'recall_scores': [0.8,
    0.8,
    0.9,
    0.9,
    0.7368421052631579,
    0.75,
    0.95,
    0.75,
    0.85,
    0.9473684210526315],
   'specifity_scores': [0.9502762430939227,
    0.9723756906077348,
    0.9668508287292817,
    0.9502762430939227,
    0.9613259668508287,
    0.9502762430939227,
    0.9779005524861878,
    0.9668508287292817,
    0.9668508287292817,
    0.9502762430939227],
   'mean_precission_score': 0.7056407867494824,
   'mean_recall_scores': 0.8384210526315788,
   'mean_specifity_scores': 0.9613259668508286},
  'SMOTE': {'precission_scores': [0.6956521739130435,
    0.8823529411764706,
    0.8181818181818182,
    0.8181818181818182,
    0.9285714285714286,
    0.7777777777777778,
    1.0,
    0.7777777777777778,
    0.85,
    0.7727272727272727],
   'recall_scores': [0.8,
    0.75,
    0.9,
    0.9,
    0.6842105263157895,
    0.7,
    0.85,
    0.7,
    0.85,
    0.8947368421052632],
   'specifity_scores': [0.9613259668508287,
    0.988950276243094,
    0.9779005524861878,
    0.9779005524861878,
    0.994475138121547,
    0.9779005524861878,
    1.0,
    0.9779005524861878,
    0.9834254143646409,
    0.9723756906077348],
   'mean_precission_score': 0.8321223008307408,
   'mean_recall_scores': 0.8028947368421052,
   'mean_specifity_scores': 0.9812154696132597},
  'ROS': {'precission_scores': [0.8888888888888888,
    0.8823529411764706,
    0.8421052631578947,
    0.7727272727272727,
    0.9166666666666666,
    0.7777777777777778,
    1.0,
    0.8235294117647058,
    0.875,
    0.8421052631578947],
   'recall_scores': [0.8,
    0.75,
    0.8,
    0.85,
    0.5789473684210527,
    0.7,
    0.8,
    0.7,
    0.7,
    0.8421052631578947],
   'specifity_scores': [0.988950276243094,
    0.988950276243094,
    0.9834254143646409,
    0.9723756906077348,
    0.994475138121547,
    0.9779005524861878,
    1.0,
    0.9834254143646409,
    0.988950276243094,
    0.9834254143646409],
   'mean_precission_score': 0.8621153485317571,
   'mean_recall_scores': 0.7521052631578947,
   'mean_specifity_scores': 0.9861878453038674},
  'BorderlineSMOTE': {'precission_scores': [0.8333333333333334,
    0.8235294117647058,
    0.8947368421052632,
    0.75,
    0.8125,
    0.7,
    0.9,
    0.875,
    0.75,
    0.7619047619047619],
   'recall_scores': [0.75,
    0.7,
    0.85,
    0.9,
    0.6842105263157895,
    0.7,
    0.9,
    0.7,
    0.75,
    0.8421052631578947],
   'specifity_scores': [0.9834254143646409,
    0.9834254143646409,
    0.988950276243094,
    0.9668508287292817,
    0.9834254143646409,
    0.9668508287292817,
    0.988950276243094,
    0.988950276243094,
    0.9723756906077348,
    0.9723756906077348],
   'mean_precission_score': 0.8101004349108063,
   'mean_recall_scores': 0.7776315789473685,
   'mean_specifity_scores': 0.9795580110497237}},
 'yeast4.dat': {'KMeansSMOTE': {'precission_scores': [0.19047619047619047,
    0.23809523809523808,
    0.3076923076923077,
    0.3684210526315789,
    0.35294117647058826,
    0.32,
    0.35714285714285715,
    0.07142857142857142,
    0.35714285714285715,
    0.2857142857142857],
   'recall_scores': [0.4,
    0.5,
    0.8,
    0.6363636363636364,
    0.6,
    0.8,
    0.5,
    0.1,
    0.45454545454545453,
    0.6],
   'specifity_scores': [0.9407665505226481,
    0.9442508710801394,
    0.9372822299651568,
    0.958041958041958,
    0.9615384615384616,
    0.9407665505226481,
    0.9686411149825784,
    0.9547038327526133,
    0.9685314685314685,
    0.9475524475524476],
   'mean_precission_score': 0.2849054536794475,
   'mean_recall_scores': 0.539090909090909,
   'mean_specifity_scores': 0.952207548549012},
  'MeansShiftSMOTE': {'precission_scores': [0.25,
    0.2631578947368421,
    0.32,
    0.3888888888888889,
    0.375,
    0.3181818181818182,
    0.42857142857142855,
    0.16666666666666666,
    0.29411764705882354,
    0.30434782608695654],
   'recall_scores': [0.4,
    0.5,
    0.8,
    0.6363636363636364,
    0.6,
    0.7,
    0.6,
    0.3,
    0.45454545454545453,
    0.7],
   'specifity_scores': [0.9581881533101045,
    0.9512195121951219,
    0.9407665505226481,
    0.9615384615384616,
    0.965034965034965,
    0.9477351916376306,
    0.9721254355400697,
    0.9477351916376306,
    0.958041958041958,
    0.9440559440559441],
   'mean_precission_score': 0.3108932170191424,
   'mean_recall_scores': 0.5690909090909091,
   'mean_specifity_scores': 0.9546441363514534},
  'SMOTE': {'precission_scores': [0.4,
    0.2857142857142857,
    0.29411764705882354,
    0.4,
    0.4166666666666667,
    0.29411764705882354,
    0.5714285714285714,
    0.0,
    0.45454545454545453,
    0.35294117647058826],
   'recall_scores': [0.4,
    0.4,
    0.5,
    0.5454545454545454,
    0.5,
    0.5,
    0.4,
    0.0,
    0.45454545454545453,
    0.6],
   'specifity_scores': [0.9790940766550522,
    0.9651567944250871,
    0.9581881533101045,
    0.9685314685314685,
    0.9755244755244755,
    0.9581881533101045,
    0.9895470383275261,
    0.9686411149825784,
    0.9790209790209791,
    0.9615384615384616],
   'mean_precission_score': 0.3469531448943214,
   'mean_recall_scores': 0.43,
   'mean_specifity_scores': 0.9703430715625837},
  'ROS': {'precission_scores': [0.6,
    0.6666666666666666,
    0.4,
    0.2,
    1.0,
    0.8,
    1.0,
    0.0,
    0.6666666666666666,
    0.5],
   'recall_scores': [0.3,
    0.2,
    0.2,
    0.09090909090909091,
    0.3,
    0.4,
    0.2,
    0.0,
    0.18181818181818182,
    0.5],
   'specifity_scores': [0.9930313588850174,
    0.9965156794425087,
    0.9895470383275261,
    0.986013986013986,
    1.0,
    0.9965156794425087,
    1.0,
    0.9895470383275261,
    0.9965034965034965,
    0.9825174825174825],
   'mean_precission_score': 0.5833333333333333,
   'mean_recall_scores': 0.2372727272727273,
   'mean_specifity_scores': 0.9930191759460051},
  'BorderlineSMOTE': {'precission_scores': [0.25,
    0.3333333333333333,
    0.26666666666666666,
    0.45454545454545453,
    0.4,
    0.46153846153846156,
    0.5714285714285714,
    0.0,
    0.4,
    0.38461538461538464],
   'recall_scores': [0.3,
    0.2,
    0.4,
    0.45454545454545453,
    0.4,
    0.6,
    0.4,
    0.0,
    0.36363636363636365,
    0.5],
   'specifity_scores': [0.9686411149825784,
    0.9860627177700348,
    0.9616724738675958,
    0.9790209790209791,
    0.9790209790209791,
    0.975609756097561,
    0.9895470383275261,
    0.975609756097561,
    0.9790209790209791,
    0.972027972027972],
   'mean_precission_score': 0.3522127872127872,
   'mean_recall_scores': 0.3618181818181818,
   'mean_specifity_scores': 0.9766233766233766}},
 'yeast5.dat': {'KMeansSMOTE': {'precission_scores': [0.5,
    0.6363636363636364,
    0.5,
    0.8888888888888888,
    0.6666666666666666,
    0.8,
    0.5,
    0.75,
    0.6,
    0.6666666666666666],
   'recall_scores': [0.8888888888888888,
    0.7777777777777778,
    0.6666666666666666,
    0.8888888888888888,
    1.0,
    0.8888888888888888,
    0.6666666666666666,
    1.0,
    1.0,
    0.75],
   'specifity_scores': [0.9722222222222222,
    0.9861111111111112,
    0.9791666666666666,
    0.9965277777777778,
    0.9861111111111112,
    0.9930555555555556,
    0.9791666666666666,
    0.9895833333333334,
    0.9791666666666666,
    0.9895833333333334],
   'mean_precission_score': 0.6508585858585858,
   'mean_recall_scores': 0.8527777777777776,
   'mean_specifity_scores': 0.9850694444444444},
  'MeansShiftSMOTE': {'precission_scores': [0.47058823529411764,
    0.6363636363636364,
    0.5,
    0.8,
    0.6666666666666666,
    0.7272727272727273,
    0.5,
    0.75,
    0.5294117647058824,
    0.6],
   'recall_scores': [0.8888888888888888,
    0.7777777777777778,
    0.6666666666666666,
    0.8888888888888888,
    1.0,
    0.8888888888888888,
    0.6666666666666666,
    1.0,
    1.0,
    0.75],
   'specifity_scores': [0.96875,
    0.9861111111111112,
    0.9791666666666666,
    0.9930555555555556,
    0.9861111111111112,
    0.9895833333333334,
    0.9791666666666666,
    0.9895833333333334,
    0.9722222222222222,
    0.9861111111111112],
   'mean_precission_score': 0.6180303030303029,
   'mean_recall_scores': 0.8527777777777776,
   'mean_specifity_scores': 0.9829861111111111},
  'SMOTE': {'precission_scores': [0.5,
    0.6666666666666666,
    0.5,
    0.8888888888888888,
    0.8,
    0.8,
    0.6,
    0.75,
    0.6428571428571429,
    0.7142857142857143],
   'recall_scores': [0.8888888888888888,
    0.6666666666666666,
    0.6666666666666666,
    0.8888888888888888,
    1.0,
    0.8888888888888888,
    0.6666666666666666,
    1.0,
    1.0,
    0.625],
   'specifity_scores': [0.9722222222222222,
    0.9895833333333334,
    0.9791666666666666,
    0.9965277777777778,
    0.9930555555555556,
    0.9930555555555556,
    0.9861111111111112,
    0.9895833333333334,
    0.9826388888888888,
    0.9930555555555556],
   'mean_precission_score': 0.6862698412698414,
   'mean_recall_scores': 0.8291666666666666,
   'mean_specifity_scores': 0.9875},
  'ROS': {'precission_scores': [0.5384615384615384,
    0.75,
    0.5714285714285714,
    1.0,
    0.875,
    0.8571428571428571,
    0.6,
    0.8181818181818182,
    0.6666666666666666,
    0.7142857142857143],
   'recall_scores': [0.7777777777777778,
    0.6666666666666666,
    0.4444444444444444,
    0.7777777777777778,
    0.875,
    0.6666666666666666,
    0.3333333333333333,
    1.0,
    0.8888888888888888,
    0.625],
   'specifity_scores': [0.9791666666666666,
    0.9930555555555556,
    0.9895833333333334,
    1.0,
    0.9965277777777778,
    0.9965277777777778,
    0.9930555555555556,
    0.9930555555555556,
    0.9861111111111112,
    0.9930555555555556],
   'mean_precission_score': 0.7391167166167166,
   'mean_recall_scores': 0.7055555555555555,
   'mean_specifity_scores': 0.992013888888889},
  'BorderlineSMOTE': {'precission_scores': [0.5333333333333333,
    0.7,
    0.5,
    0.8888888888888888,
    0.7272727272727273,
    0.8,
    0.5714285714285714,
    0.75,
    0.6,
    0.7142857142857143],
   'recall_scores': [0.8888888888888888,
    0.7777777777777778,
    0.6666666666666666,
    0.8888888888888888,
    1.0,
    0.8888888888888888,
    0.4444444444444444,
    1.0,
    1.0,
    0.625],
   'specifity_scores': [0.9756944444444444,
    0.9895833333333334,
    0.9791666666666666,
    0.9965277777777778,
    0.9895833333333334,
    0.9930555555555556,
    0.9895833333333334,
    0.9895833333333334,
    0.9791666666666666,
    0.9930555555555556],
   'mean_precission_score': 0.6785209235209234,
   'mean_recall_scores': 0.8180555555555555,
   'mean_specifity_scores': 0.9875}}}

In [15]:
from scipy.stats import ttest_rel, wilcoxon, shapiro
from tabulate import tabulate

alfa = .05
methods = ["KMeansSMOTE", "MeansShiftSMOTE", "SMOTE", "ROS", "BorderlineSMOTE"]
metrics = ["precission_scores","recall_scores","specifity_scores"]

for directory in results:
    for metric in metrics:
        w_statistic = np.zeros((len(methods), len(methods)))
        p_value = np.zeros((len(methods), len(methods)))
        test_used = np.empty((len(methods), len(methods)), dtype=object)
        for i, preprocessing_method in enumerate(methods):
            for j, comparison_preprocessing_method in enumerate(methods):
                metric_results_one = results[directory][preprocessing_method][metric]
                metric_results_two = results[directory][comparison_preprocessing_method][metric]
                tmp_p_val = 0
                try:
                    _, tmp_p_val = wilcoxon(metric_results_one, metric_results_two)
                except: 
                    _, tmp_p_val = 0, 1
                mean_metric_one = np.mean(metric_results_one)
                mean_metric_two = np.mean(metric_results_two)
                if  tmp_p_val <= alfa:
                    if mean_metric_one - mean_metric_two > 0:
                        w_statistic[i,j], p_value[i,j] = 1, 1
                    else:
                        w_statistic[i,j], p_value[i,j] = 0, 1
                else:
                    if mean_metric_one - mean_metric_two > 0:
                        w_statistic[i,j], p_value[i,j] = 1, 0
                    else:
                        w_statistic[i,j], p_value[i,j] = 0, 0
        stat_better = w_statistic * p_value
        stat_better_table = tabulate(stat_better, methods)
        print(f"Statistically significantly better {metric}:")
        print(stat_better_table)
        print()
        print()
        print()
        

Statistically significantly better precission_scores:
  KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
-------------  -----------------  -------  -----  -----------------
            0                  1        0      1                  0
            0                  0        0      1                  0
            1                  1        0      0                  0
            1                  1        0      0                  0
            1                  1        0      0                  0

Statistically significantly better recall_scores:
  KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
-------------  -----------------  -------  -----  -----------------
            0                  0        1      1                  1
            0                  0        1      1                  1
            0                  0        0      1                  0
            0                  0        0      0                  0
           

c:\Users\JG\Documents\GitHub\SMOTE-clustering\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning:

scipy.stats.shapiro: Input data has range zero. The results may not be accurate.



Statistically significantly better precission_scores:
  KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
-------------  -----------------  -------  -----  -----------------
            0                  0        0      0                  0
            0                  0        0      0                  0
            0                  0        0      0                  0
            0                  0        0      0                  0
            0                  0        0      0                  0

Statistically significantly better recall_scores:
  KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
-------------  -----------------  -------  -----  -----------------
            0                  0        0      0                  0
            0                  0        0      0                  0
            0                  0        0      0                  0
            0                  0        0      0                  0
           

### Statistically significantly better preprocessings for all datasets

In [17]:
from scipy.stats import rankdata, ranksums

methods = ["KMeansSMOTE", "MeansShiftSMOTE", "SMOTE", "ROS", "BorderlineSMOTE"]
metrics = ["mean_precission_score", "mean_recall_scores", "mean_specifity_scores"]
for metric in metrics:
    mean = []
    for directory in results:
        preprocessing_mean = []
        for i, preprocessing_method in enumerate(methods):
            preprocessing_mean.append(results[directory][preprocessing_method][metric])
        mean.append(preprocessing_mean)

    ranks = []
    for mean_score in mean:
        ranks.append(rankdata(mean_score).tolist())
    ranks = np.array(ranks)

    alfa = .05
    w_statistic = np.zeros((len(methods), len(methods)))
    p_value = np.zeros((len(methods), len(methods)))
    for i in range(len(methods)):
        for j in range(len(methods)):
            w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])
    names_column = np.expand_dims(np.array(list(methods)), axis=1)
    w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
    w_statistic_table = tabulate(w_statistic_table, methods, floatfmt=".2f")
    p_value_table = np.concatenate((names_column, p_value), axis=1)
    p_value_table = tabulate(p_value_table, methods, floatfmt=".2f")
    advantage = np.zeros((len(methods), len(methods)))
    advantage[w_statistic > 0] = 1
    advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), methods)
    significance = np.zeros((len(methods), len(methods)))
    significance[p_value <= alfa] = 1
    statisticaly_better = advantage * significance
    statisticaly_better_table = tabulate(np.concatenate(
    (names_column, statisticaly_better), axis=1), methods)
    print(f"Metric: {metric}")
    print("Statistical significance (alpha = 0.05):")
    print(statisticaly_better_table)
    print()
    print()

Metric: mean_precission_score
Statistical significance (alpha = 0.05):
                   KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
---------------  -------------  -----------------  -------  -----  -----------------
KMeansSMOTE                  0                  0        0      0                  0
MeansShiftSMOTE              0                  0        0      0                  0
SMOTE                        1                  1        0      0                  0
ROS                          1                  1        0      0                  0
BorderlineSMOTE              1                  1        0      0                  0


Metric: mean_recall_scores
Statistical significance (alpha = 0.05):
                   KMeansSMOTE    MeansShiftSMOTE    SMOTE    ROS    BorderlineSMOTE
---------------  -------------  -----------------  -------  -----  -----------------
KMeansSMOTE                  0                  0        1      1                  1
MeansShif

# Testy parowe
## Cel testów parowych
W poprzedniej części pracy wykonano ocenę klasyfikatorów za pomocą metryk i walidacji krzyżowej. W ten sposób uzyskano osobną ocenę dla przypadków klasyfikatorów gdzie różnicami były: zbiory uczące, różne modele uczenia maszynowego oraz główny cel pracy czyli różne metody balansowania danych (sprawdzić czy niema zmian). Uzyskane wyniki należy w tym momencie porównać aby ocenić czy wyniki uzyskane w przypadkach gdzie wykorzystano metodę SMOTE wykonywaną na klastrach powstałych z danych niezbalansowanych są lepsze od pozostałych metod oversamplingu. Aby ocenić tą zależność można uśrednić uzyskane metryki i ocenić czy średnio model wykorzystujący zaproponowaną metodę oversamplingu uzyskuje lepsze wyniki, jednak test taki nie może zostać uznany za prawidłowy ponieważ mógł wynikać z przypadku. W tym celu należy wziąć pod uwagę również parametr określający jak wyniki wchodzące w skład średniej są od niej oddalone (odchylenie standardowe zbiorów). W tym celu przez badaczy wykorzystywane są testy statystyczne takie jak test T- Studenta oraz test Wilcoxona. Poniżej opisane zostaną testy które mogą zostać wykorzystane w pracy.
## Wkorzystane testy statystyczne 
Testy statystyczne wykorzystuje się dla różńych zbiorów danych aby ocenić czy różnica między nimi jest statystycznie istotna.
W tej pracy celem jest porównanie metryk dla modeli wykorzystujących mechanizm balansowania danych opartych o metodę SMOTE działającą na pojedynczych klastrach danych wejściowych oraz innych popularnie wykorzystywanych narzędzi oversamplingu. W ten sposób dane można podzielić na pary: zaproponowana w pracy metoda i inna metoda oversamplingu. Taki sposób podziału danych determinuje wykorzystanie testów parowych.
### Test T-Studenta
Test T-Studenta to test parametryczny (opierający się o porównanie parametrów populacji takich jak odchylenie standardowe czy średnia )
Warunkiem koniecznym do zastosowania testu T studenta jest założenie, że porównywane zbiory są normalne. Testowanie normalności zbiorów zostanie omówione w dalszej części pracy. Jeśli Testowanie normalności wykazało, że próbki nie są normalne wtedy można wykorzystać inne testy o mniejszej precyzji czyli testy nieparametryczne. Należy jednak napiętać, że jeśli to możliwe powinienny zostać przeprowadzone testy parametryczne takie jest test T studenta lub analiza wariancji.\
Testy statystyczne parametryczne : https://pogotowiestatystyczne.pl/slowniki/testy-parametryczne/#:~:text=Testy%20parametryczne%20to%20rodzaj%20test%C3%B3w,standardowe%20lub%20innych%20statystykach%20opisowych.
### Test Wilcoxona
Test Wilcoxona to test nieparametryczny wykonywany na bazie próbek populacji a nie na jej parametrach. Wykorzystuje on różnicę między próbkami w przypadkach wykorzystania dwóchróżnych hiperparametrów modelu. Różnica każdejz próbek zostaje zakwalifikowana do jednego ze zbiorów $T_{-}$ gdy różnica jest ujemna lub $T_{+}$ gdy jest dodatnia. W ten sposób uzyskano dwa zbiory. Wszystkim różnicom w tym momencie usuwany zostaje znak a przyznana zostaje  $ranga$(tu można walnąć dokładniejszy opis ale teraz trochę małoczasu) a następnie sumowane są wszystkie ranki w zbiorach $T_{-}$ i $T_{+}$. Pod uwagę bierze się mniejszą sumę rang oraz sprawdzana jest ona w tablicy wartości sum wag Wilxocona.
### Test założenia Normalności
Aby ocenić czy próbki pewnej populacji mają określony rozkład można wykorzystać Test Kołmogorowa-Smirnowa. Aby określić czy wyniki badań możemy ocenić za pomocą testu T studenta trzeba spełnić założenie normalności a więc określić czy próbki mogą pochodzić z wałsciwości o charakterze rozkładu normalngo. W tym celu test Kołmogorowa-Smirnowa należy wykonać dla uzyskanych wcześniej wyników i rozkładu normalnego.

biblio: https://www.scirp.org/html/6-1241391_107034.htm
https://onlinelibrary.wiley.com/doi/full/10.1002/9781118445112.stat06558

Do wykonania oceny założenia normalności wykorzystana zostanie funkcja kstest pochodząca z pakiety scipy \
Tutaj jak to robić: https://medium.com/@ricardojaviermartnezsustegui/kolmog%C3%B3rov-smirnov-test-in-python-step-by-step-1b7532021bd2

a jest jeszcze coś takiego
scipy.stats.normaltest